# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages, which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission if necessary. 

> **Note**: Once you have completed all of the code implementations, you need to finalize your work by exporting the iPython Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You can then export the notebook by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission. 

In addition to implementing code, there is a writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) that can be used to guide the writing process. Completing the code template and writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/481/view) for this project.

The [rubric](https://review.udacity.com/#!/rubrics/481/view) contains "Stand Out Suggestions" for enhancing the project beyond the minimum requirements. The stand out suggestions are optional. If you decide to pursue the "stand out suggestions", you can include the code in this Ipython notebook and also discuss the results in the writeup file.


>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---
## Step 0: Load The Data

In [ ]:
# Load the pickled data
import pickle

training_file = "./data/train.p"
validation_file="./data/valid.p"
testing_file = "./data/test.p"

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train_initial, y_train_initial = train['features'], train['labels']
X_valid_initial, y_valid = valid['features'], valid['labels']
X_test_initial, y_test = test['features'], test['labels']


---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 1D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below. Use python, numpy and/or pandas methods to calculate the data summary rather than hard coding the results. For example, the [pandas shape method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.shape.html) might be useful for calculating some of the summary results. 

### Provide a Basic Summary of the Data Set Using Python, Numpy and/or Pandas

In [ ]:
import numpy as np

# Determine the number of training examples

assert(len(X_train_initial) == len(y_train_initial))
n_train = len(X_train_initial)

# Determine the number of validation examples
assert(len(X_valid_initial) == len(y_valid))
n_validation = len(X_valid_initial)

# Determine the number of testing examples.
assert(len(X_test_initial) == len(y_test))
n_test = len(X_test_initial)

# Determine the shape of an traffic sign image?
image_shape = X_train_initial[0].shape

# Determine now many unique classes/labels there are in the dataset.
n_classes = len(np.unique(y_train_initial))

print("Number of training samples =", n_train)
print("Number of validation samples =", n_validation)
print("Number of testing samples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

### Include an exploratory visualization of the dataset

Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc. 

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections. It can be interesting to look at the distribution of classes in the training, validation and test set. Is the distribution the same? Are there more examples of some classes than others?

In [ ]:
### Data exploration visualization code goes here.
### Feel free to use as many code cells as needed.
import matplotlib.pyplot as plt
import pandas as pd
import random
sign_names = pd.read_csv("signnames.csv").values[:,1]

# Visualizations will be shown in the notebook.
%matplotlib inline

fig = plt.figure(figsize=(32,32))
for i in range(n_classes):
    k = np.where(y_train_initial==i)
    indx=random.randint(0, (len(k[0]-1)))
    img = X_train_initial[k[0][indx]]
    subfig = fig.add_subplot(int(n_classes/5)+1,5,i+1) 
    subfig.imshow(img)
    subfig.set_title("{}: {}".format(i,sign_names[i]))
fig.tight_layout()
fig.show()

def showdatabarchart(y_data,title):
    sign_types, test_counts=np.unique(y_data,return_counts=True)
    fig=plt.figure(figsize=(20,10))
    plt.barh( np.arange( n_classes ), test_counts, align='center')
    plt.title(title)
    plt.ylabel('Class')
    plt.yticks(np.arange(len(sign_names)), sign_names)
    plt.xlabel('Number of Samples')
    plt.show()
    return fig

fig=showdatabarchart(y_train_initial,"Training Data")
fig.savefig('./visualizations/TrainingData.jpg')
fig=showdatabarchart(y_valid,"Validation Data")
fig.savefig('./visualizations/ValidationData.jpg')
fig=showdatabarchart(y_test,"Test Data")
fig.savefig('./visualizations/TestData.jpg')

### Augment Data
From reading several papers on the subject it was clear that data augmentation is a common way to increase accuracy. The cell below reads the initially loaded training data and doubles the size of it by apply random: shifts, rotations, shears, and zooms using the Keras image data generator.

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
# Visualizations will be shown in the notebook.
%matplotlib inline
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
datagen = ImageDataGenerator(
        rotation_range=15,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=False,
        dim_ordering='tf',
        fill_mode='nearest')
for X_batch, y_batch in datagen.flow(X_train_initial, y_train_initial, batch_size=X_train_initial.shape[0], shuffle=False):
    X_train_augmented = X_batch.astype('uint8')
    y_train_augmented = y_batch
    break
    
X_train_after_augmentation = np.concatenate((X_train_initial, X_train_augmented), axis=0)
y_train_after_augmentation = np.concatenate((y_train_initial, y_train_augmented), axis=0)

assert(len(X_train_after_augmentation) == len(y_train_after_augmentation))
# reset n_train
n_train_before_augmentation=len(X_train_initial)
n_train = len(X_train_after_augmentation)

index = random.randint(0, (len(X_train_after_augmentation)/2-1))

def showgeneratedimage(index):
    image1 = X_train_after_augmentation[index]
    image2 = X_train_after_augmentation[index+n_train_before_augmentation]
    plt.figure(figsize=(1,1))
    plt.imshow(image1)
    plt.figure(figsize=(1,1))
    plt.imshow(image2)

# Show a few images and their randomly generated equivalents    
num_images_to_show=4
fig = plt.figure(figsize=(12,8))
for i in range(num_images_to_show):
    index = random.randint(0, len(X_train_after_augmentation)/2)
    orig_img=X_train_after_augmentation[index];
    processed_img=X_train_after_augmentation[index+n_train_before_augmentation]
    ax = fig.add_subplot(num_images_to_show,2,2*i+1) 
    ax.imshow(orig_img)
    ax.set_title("Image at Index:{} (Before)".format(index))
    ax = fig.add_subplot(num_images_to_show,2,2*i+2) 
    ax.imshow(processed_img)
    ax.set_title("Image at Index:{} (After)".format(index))
fig.tight_layout()
fig.savefig('./visualizations/AugmentedData.jpg')

----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play! 

With the LeNet-5 solution from the lecture, you should expect a validation set accuracy of about 0.89. To meet specifications, the validation set accuracy will need to be at least 0.93. It is possible to get an even higher accuracy, but 0.93 is the minimum for a successful project submission. 

There are various aspects to consider when thinking about this problem:

- Neural network architecture (is the network over or underfitting?)
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

### Pre-process the Data Set (normalization, grayscale, etc.)

I preprocessed the images as follows: 
(1) Convert to grayscale. Accuracy was a 2 or 3 percent higher with grayscale and ran faster also because there is only a single channel. I assume the acuracy was higher because shape and other factors were more distinct than color between the sign types and eliminating color allowed the network to focus on these other factors.
(2) Improve the contract by calling equalizeHist
(3) Normalized the images using (x-avg)/stddev

In [ ]:
### Preprocess the data here. It is required to normalize the data. Other preprocessing steps could include 
### converting to grayscale, etc.
### Feel free to use as many code cells as needed.
import cv2
from sklearn.utils import shuffle
X_train_after_augmentation, y_train = shuffle(X_train_after_augmentation, y_train_after_augmentation)

def initializegrayscaleimagearrayfromRGB(arrayRGB):
    data_shape=arrayRGB.shape
    data_shape2=(data_shape[0],data_shape[1],data_shape[2],1)
    return np.empty(shape=data_shape2)

def normalizeimages(imgs):
    imgs=imgs.astype('float32')    
    avg=np.mean(imgs) 
    sigma=np.std(imgs)
    imgs=(imgs-avg)/sigma
    return imgs

def preprocessimages(rgbimages):
    grayscaleimages=initializegrayscaleimagearrayfromRGB(rgbimages)
    nimages=len(rgbimages)
    for i in range(nimages):
        img=cv2.cvtColor(rgbimages[i], cv2.COLOR_RGB2GRAY)
        img=cv2.equalizeHist(img)
        grayscaleimages[i]=img.reshape(32,32,1)
    return normalizeimages(grayscaleimages)

X_train=preprocessimages(X_train_after_augmentation)
X_valid=preprocessimages(X_valid_initial)
X_test=preprocessimages(X_test_initial)

num_images_to_show=4
fig = plt.figure(figsize=(9,6))
for i in range(num_images_to_show):
    index = random.randint(0, n_train-1)
    if index==n_train :
        index-=1
    orig_img=X_train_after_augmentation[index]
    processed_img=X_train[index]
    ax = fig.add_subplot(num_images_to_show,2,2*i+1) 
    ax.imshow(orig_img)
    ax.set_title("Image at Index:{} (Before)".format(index))
    ax = fig.add_subplot(num_images_to_show,2,2*i+2)
    ax.imshow(processed_img.squeeze(),cmap="gray")
    ax.set_title("Image at Index:{} (After)".format(index))
fig.tight_layout()
fig.savefig('./visualizations/ProcessedData.jpg')

### Model Architecture

In [ ]:
### Define your architecture here.
### Feel free to use as many code cells as needed.
import tensorflow as tf

EPOCHS = 80
BATCH_SIZE = 128

In [ ]:
from tensorflow.contrib.layers import flatten
def LeNet(x):    
    # Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer
    mu = 0
    sigma = 0.1
    weights = {
    'layer1': tf.Variable(tf.random_normal([5, 5, 1, 32],mean = mu, stddev = sigma)),
    'layer2': tf.Variable(tf.random_normal([5, 5, 32, 64],mean = mu, stddev = sigma)),
    'layer3': tf.Variable(tf.random_normal([1600, 120],mean = mu, stddev = sigma)),
    'layer4': tf.Variable(tf.random_normal([120, 84],mean = mu, stddev = sigma)),
    'layer5': tf.Variable(tf.random_normal([84, 43],mean = mu, stddev = sigma))}

    biases = {
    'layer1': tf.Variable(tf.zeros(32)),
    'layer2': tf.Variable(tf.zeros(64)),
    'layer3': tf.Variable(tf.zeros(120)),
    'layer4': tf.Variable(tf.zeros(84)),
    'layer5': tf.Variable(tf.zeros(43))}
    
    # TODO: Layer 1: Convolutional. Input = 32x32x1. Output = 28x28x32.
    l1output=tf.nn.conv2d(x,weights['layer1'],strides=[1,1,1,1],padding='VALID')
    l1output = tf.nn.bias_add(l1output, biases['layer1'])
#    print("l1output={}".format(l1output.get_shape()))
    # TODO: Activation.
    l1output=tf.nn.relu(l1output)

    # TODO: Pooling. Input = 28x28x32. Output = 14x14x32.
    l1output = tf.nn.max_pool(
    l1output,
    ksize=[1, 2, 2, 1],
    strides=[1, 2, 2, 1],
    padding='VALID')
#    print("l1output second={}".format(l1output.get_shape()))

    # TODO: Layer 2: Convolutional. Output = 10x10x64.
    l2output=tf.nn.conv2d(l1output,weights['layer2'],strides=[1,1,1,1],padding='VALID')
    l2output = tf.nn.bias_add(l2output, biases['layer2'])
    # TODO: Activation.
    l2output=tf.nn.relu(l2output)
#    print("l2output first={}".format(l2output.get_shape()))
    # TODO: Pooling. Input = 10x10x64. Output = 5x5x64.
    l2output = tf.nn.max_pool(
    l2output,
    ksize=[1, 2, 2, 1],
    strides=[1, 2, 2, 1],
    padding='VALID')
#    print("l2output second={}".format(l2output.get_shape()))
    # TODO: Flatten. Input = 5x5x64. Output = 1600.
# I tried using an architecture where the output of two convolutional layers are fed
# into the first fully-connected layer, but it did not improve accuracy for me
#    conv_output=tf.concat(1,[flatten(l2output),flatten(l1output)])
    conv_output=flatten(l2output)
    # TODO: Layer 3: Fully Connected. Input = 1600. Output = 120.
    l3output   = tf.add(tf.matmul(conv_output, weights['layer3']),biases['layer3'])
    # TODO: Activation.
    l3output=tf.nn.relu(l3output)
    l3output = tf.nn.dropout(l3output, keep_prob)
    # TODO: Layer 4: Fully Connected. Input = 120. Output = 84.
    l4output   = tf.add(tf.matmul(l3output, weights['layer4']),biases['layer4'])
    # TODO: Activation.
    l4output=tf.nn.relu(l4output)
    l4output = tf.nn.dropout(l4output, keep_prob)
    # TODO: Layer 5: Fully Connected. Input = 84. Output = 43.
    logits   = tf.add(tf.matmul(l4output, weights['layer5']),biases['layer5'])
    return logits

In [ ]:
# Define placeholders
x = tf.placeholder(tf.float32, (None, 32, 32,1))
y = tf.placeholder(tf.int32, (None))
keep_prob=tf.placeholder(tf.float32)
one_hot_y = tf.one_hot(y, 43)

### Train, Validate and Test the Model

A validation set can be used to assess how well the model is performing. A low accuracy on the training and validation
sets imply underfitting. A high accuracy on the training set but low accuracy on the validation set implies overfitting.

In [ ]:
### Train your model here.
### Calculate and report the accuracy on the training and validation set.
### Once a final model architecture is selected, 
### the accuracy on the test set should be calculated and reported as well.
### Feel free to use as many code cells as needed.
rate = 0.0005

logits = LeNet(x)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y, logits=logits)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)

In [ ]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y, keep_prob: 1.0})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

In [ ]:
import datetime as dt
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train)
    
    print("Training...")
    print()
    
    n1=dt.datetime.now()


    for i in range(EPOCHS):
        n3=dt.datetime.now()
        X_train, y_train = shuffle(X_train, y_train)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y, keep_prob: 0.6})
            
        validation_accuracy = evaluate(X_valid, y_valid)
        print("EPOCH {} ...".format(i+1))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        n4=dt.datetime.now()
        print("Elapsed time for EPOCH={} seconds".format((n4-n3).seconds))
        print()
    n2=dt.datetime.now()
    print("Elapsed time={} seconds".format((n2-n1).seconds))
    training_accuracy=evaluate(X_train, y_train)
    print("Training Accuracy = {:.3f}".format(training_accuracy))
    saver.save(sess, './lenet')
    print("Model saved")

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))

    test_accuracy = evaluate(X_test, y_test)
    print("Test Accuracy = {:.3f}".format(test_accuracy))

---

## Step 3: Test a Model on New Images

To give yourself more insight into how your model is working, download at least five pictures of German traffic signs from the web and use your model to predict the traffic sign type.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Load and Output the Images

In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.
import os
import numpy as np
import csv
reader = csv.DictReader(open('signlabels.csv'))
newdatalabels = {}
for row in reader:
    newdatalabels[row['FileName']]=row['Label']
    
X_new = list()
y_new = np.array([], dtype=np.int)
imagedir = './downloadedgermansigns'
for imagePath in os.listdir(imagedir):
    if imagePath.find(".jpg") != -1: 
        image = cv2.imread(os.path.join(imagedir, imagePath))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        fileparts = imagePath.split('.')
        filename=fileparts[0]
        image = cv2.resize(image, (32, 32),interpolation = cv2.INTER_AREA)
        X_new.append(image)
        y_new=np.append(y_new,newdatalabels[filename])

In [ ]:
import matplotlib.pyplot as plt
# Visualizations will be shown in the notebook.
%matplotlib inline
    
X_new_processed = preprocessimages(np.asarray(X_new))

num_new_images=len(X_new_processed)
fig = plt.figure(figsize=(8,16))
for i in range(num_new_images):
    orig_img=X_new[i];
    processed_img=X_new_processed[i]
    ax = fig.add_subplot(num_new_images,2,2*i+1) 
    ax.imshow(orig_img)
    ax.set_title("Label: {}".format(y_new[i]))
    ax = fig.add_subplot(num_new_images,2,2*i+2) 
    ax.imshow(processed_img.squeeze(),cmap="gray")
    ax.set_title("Label: {}".format(y_new[i]))
plt.tight_layout()

### Predict the Sign Type for Each Image

In [ ]:
### Run the predictions here and use the model to output the prediction for each image.
### Make sure to pre-process the images with the same pre-processing pipeline used earlier.
### Feel free to use as many code cells as needed.
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))
    n5=dt.datetime.now()
    test_accuracy = evaluate(X_new_processed, y_new)
    n6=dt.datetime.now()
    print("Elapsed time={} seconds".format((n6-n5).seconds))
    

### Analyze Performance

In [ ]:
### Calculate the accuracy for these 5 new images. 
### For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate on these new images.
print("Test Accuracy = {:.3f}".format(test_accuracy))

### Output Top 5 Softmax Probabilities For Each Image Found on the Web

For each of the new images, print out the model's softmax probabilities to show the **certainty** of the model's predictions (limit the output to the top 5 probabilities for each image). [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. 

The example below demonstrates how tf.nn.top_k can be used to find the top k predictions for each image.

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example. The values in the array represent predictions. The array contains softmax probabilities for five candidate images with six possible classes. `tk.nn.top_k` is used to choose the three classes with the highest probability:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

In [ ]:
### Print out the top five softmax probabilities for the predictions on the German traffic sign images found on the web. 
### Feel free to use as many code cells as needed.
import matplotlib.pyplot as plt
# feed_dict_new = feed_dict=
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))
    top_5_prob = sess.run(tf.nn.top_k(tf.nn.softmax(logits),5),feed_dict = {x: X_new_processed, y: y_new, keep_prob: 1.0})

fig = plt.figure(figsize=(8,24))
for i in range(num_new_images):
    plt = fig.add_subplot(num_new_images,2,2*i+1) 
    plt.axis('off')
    plt.imshow(X_new_processed[i].squeeze(),cmap='gray')
    
    plt=fig.add_subplot(num_new_images,2,2*i+2)
    plt.barh(5-np.arange(5),top_5_prob[0][i])
    for i_lbl in range(5): 
      plt.text(top_5_prob[0][i][0]+.1,5-i_lbl-.10,sign_names[top_5_prob[1][i][i_lbl]])
print(top_5_prob[0][0])
print(top_5_prob[1][0])
fig.savefig('./visualizations/top5probs.jpg')

### Project Writeup

Once you have completed the code implementation, document your results in a project writeup using this [template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) as a guide. The writeup can be in a markdown or pdf file. 

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

---

## Step 4 (Optional): Visualize the Neural Network's State with Test Images

 This Section is not required to complete but acts as an additional excersise for understaning the output of a neural network's weights. While neural networks can be a great learning device they are often referred to as a black box. We can understand what the weights of a neural network look like better by plotting their feature maps. After successfully training your neural network you can see what it's feature maps look like by plotting the output of the network's weight layers in response to a test stimuli image. From these plotted feature maps, it's possible to see what characteristics of an image the network finds interesting. For a sign, maybe the inner network feature maps react with high activation to the sign's boundary outline or to the contrast in the sign's painted symbol.

 Provided for you below is the function code that allows you to get the visualization output of any tensorflow weight layer you want. The inputs to the function should be a stimuli image, one used during training or a new one you provided, and then the tensorflow variable name that represents the layer's state during the training process, for instance if you wanted to see what the [LeNet lab's](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) feature maps looked like for it's second convolutional layer you could enter conv2 as the tf_activation variable.

For an example of what feature map outputs look like, check out NVIDIA's results in their paper [End-to-End Deep Learning for Self-Driving Cars](https://devblogs.nvidia.com/parallelforall/deep-learning-self-driving-cars/) in the section Visualization of internal CNN State. NVIDIA was able to show that their network's inner weights had high activations to road boundary lines by comparing feature maps from an image with a clear path to one without. Try experimenting with a similar test to show that your trained network's weights are looking for interesting features, whether it's looking at differences in feature maps from images with or without a sign, or even what feature maps look like in a trained network vs a completely untrained one on the same sign image.

<figure>
 <img src="visualize_cnn.png" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above)</p> 
 </figcaption>
</figure>
 <p></p> 


In [ ]:
### Visualize your network's feature maps here.
### Feel free to use as many code cells as needed.

# image_input: the test image being fed into the network to produce the feature maps
# tf_activation: should be a tf variable name used during your training procedure that represents the calculated state of a specific weight layer
# activation_min/max: can be used to view the activation contrast in more detail, by default matplot sets min and max to the actual min and max values of the output
# plt_num: used to plot out multiple different weight feature map sets on the same block, just extend the plt number for each new feature map entry

def outputFeatureMap(image_input, tf_activation, activation_min=-1, activation_max=-1 ,plt_num=1):
    # Here make sure to preprocess your image_input in a way your network expects
    # with size, normalization, ect if needed
    # image_input =
    # Note: x should be the same name as your network's tensorflow data placeholder variable
    # If you get an error tf_activation is not defined it may be having trouble accessing the variable from inside a function
    activation = tf_activation.eval(session=sess,feed_dict={x : image_input})
    featuremaps = activation.shape[3]
    plt.figure(plt_num, figsize=(15,15))
    for featuremap in range(featuremaps):
        plt.subplot(6,8, featuremap+1) # sets the number of feature maps to show on each row and column
        plt.title('FeatureMap ' + str(featuremap)) # displays the feature map number
        if activation_min != -1 & activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin =activation_min, vmax=activation_max, cmap="gray")
        elif activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmax=activation_max, cmap="gray")
        elif activation_min !=-1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin=activation_min, cmap="gray")
        else:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", cmap="gray")